<a href="https://colab.research.google.com/github/jessilver/Organizacao_Computadores/blob/develop/Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Faça uma cópia no seu drive

Importante: as instalações precisam ser feita toda vez que for iniciada uma nova seção no Colab

# Jessé Eliseu Nunes da Silva

# Interface

## Código

In [ ]:
# ===================================================================
# Bloco 1: Interface do Usuário
# Define os elementos visuais (widgets) e as funções (handlers)
# que respondem às interações do usuário (cliques de botão).
# Este bloco configura a aparência e o controle geral do simulador.
# ===================================================================

import ipywidgets as widgets
from IPython.display import display, clear_output

class LogColors: INFO, SUCCESS, ERROR, WARNING, RESET = ('\033[94m', '\033[92m', '\033[91m', '\033[93m', '\033[0m')

# --- Widgets ---
# Criação dos componentes visuais da interface.

# Área para inserir o código Assembly
code_area = widgets.Textarea(value="", placeholder="Escreva seu código assembly aqui...", description="Code:", layout=widgets.Layout(width="100%", height="200px"))
# Área onde os resultados (tabelas, estatísticas) serão exibidos
output_area = widgets.Output()

# Seletores para configurar a simulação
part_selector = widgets.RadioButtons(options=[('Ciclo Único', 'p1_2'), ('Pipeline (Completo)', 'p_extra')], description='Simular:')
hazard_mode_selector = widgets.RadioButtons(options=[('Sem tratamento de hazards', 'none'), ('Stall em hazards', 'stall'), ('Forwarding', 'forwarding')], value='forwarding', description='Hazards:')
cache_selector = widgets.RadioButtons(options=[('Com Cache', 'enabled'), ('Sem Cache', 'disabled')], value='enabled', description='Memória:')
cache_policy_selector = widgets.RadioButtons(options=[('LRU', 'lru'), ('Random', 'random')], value='lru', description='Política Cache:')
cache_penalty_input = widgets.IntText(value=10, description="Penalidade Miss:")

# --- Botões de Ação ---
load_button = widgets.Button(description="Load/Reset", button_style="primary")
step_single_cycle_button = widgets.Button(description="Próximo Passo ->", button_style="success", icon='step-forward')
step_pipeline_button = widgets.Button(description="Próximo Ciclo ->", button_style="success", icon='step-forward')
run_all_button = widgets.Button(description="Run All", button_style="info", icon='fast-forward')

reg_button = widgets.Button(description="Show Registers", button_style="info")
mem_button = widgets.Button(description="Show Memory", button_style="info")
cache_button = widgets.Button(description="Show Cache Stats", button_style="warning")
start_addr = widgets.IntText(value=0, description="Start Addr")
mem_range = widgets.IntText(value=10, description="Range")

# --- Handlers ---
# Funções que são executadas quando os botões são clicados.

def load_program_handler(b):
    """
    Handler para o botão 'Load/Reset'.
    Limpa a interface e chama a função de inicialização apropriada
    (do Bloco 9) baseada no modo selecionado.
    """
    # Limpa o log principal da célula e a área de output do widget
    clear_output(wait=True)
    display(ui) # Redesenha a UI
    with output_area:
        clear_output(wait=True)

    # Chama a função de inicialização correta
    if part_selector.value == 'p1_2':
        initialize_single_cycle_simulation() # Definida no Bloco 9
    else: # Pipeline
        initialize_pipeline_simulation() # Definida no Bloco 9

def step_single_cycle_handler(b):
    """Handler para o botão 'Próximo Passo ->' (Ciclo Único)."""
    execute_one_single_cycle_step() # Definida no Bloco 9

def step_pipeline_handler(b):
    """Handler para o botão 'Próximo Ciclo ->' (Pipeline)."""
    execute_one_pipeline_cycle() # Definida no Bloco 9

def run_all_handler(b):
    """
    Handler para o botão 'Run All'.
    Primeiro, reseta e carrega o programa usando as configurações atuais,
    depois executa todas as etapas (passos ou ciclos) até o fim.
    """
    load_program_handler(None) # Garante reset com configs atuais
    print(f"\n{LogColors.INFO}--- Executando todas as etapas ---{LogColors.RESET}")
    if part_selector.value == 'p1_2':
        # Chama a função de passo repetidamente até ela retornar True (fim)
        while not execute_one_single_cycle_step():
            pass
    else: # Pipeline
        while not execute_one_pipeline_cycle():
            pass

def reg_button_handler(b):
    """Handler para o botão 'Show Registers'."""
    show_registers(regs) # Definida no Bloco 2

def mem_button_handler(b):
    """Handler para o botão 'Show Memory'."""
    show_memory(mem, start=start_addr.value, rng=mem_range.value) # Definida no Bloco 2

def cache_button_handler(b):
    """Handler para o botão 'Show Cache Stats'."""
    if part_selector.value != 'p_extra' or cache_selector.value == 'disabled':
        with output_area:
            clear_output(wait=True)
            print(f"{LogColors.WARNING}A cache não está ativa. As estatísticas não são aplicáveis.{LogColors.RESET}")
    else:
        show_cache_stats(cache) # Definida no Bloco 2

# --- Lógica de UI Dinâmica ---
# Funções que mostram/ocultam partes da interface dependendo das seleções.

def on_cache_selection_change(change):
    """Mostra/oculta as opções de política e penalidade da cache."""
    is_cache_enabled = (change['new'] == 'enabled')
    cache_policy_selector.layout.display = 'flex' if is_cache_enabled else 'none'
    cache_penalty_input.layout.display = 'flex' if is_cache_enabled else 'none'

def on_part_selection_change(change):
    """Mostra/oculta todas as opções dependentes do modo Pipeline."""
    is_pipeline = (change['new'] == 'p_extra')
    # Alterna a visibilidade dos botões de passo
    step_single_cycle_button.layout.display = 'inline-block' if not is_pipeline else 'none'
    step_pipeline_button.layout.display = 'inline-block' if is_pipeline else 'none'
    # Mostra/oculta todas as opções de pipeline
    for widget in [hazard_mode_selector, cache_selector, cache_policy_selector, cache_penalty_input, cache_button]:
        widget.layout.display = 'flex' if is_pipeline else 'none'
    # Garante que as sub-opções da cache estejam no estado correto
    if is_pipeline: on_cache_selection_change({'new': cache_selector.value})
    else: cache_policy_selector.layout.display = 'none'; cache_penalty_input.layout.display = 'none'

part_selector.observe(on_part_selection_change, names='value')
cache_selector.observe(on_cache_selection_change, names='value')

# --- Conexões e Layout ---
# Associa cada botão ao seu handler correspondente.
load_button.on_click(load_program_handler)
step_single_cycle_button.on_click(step_single_cycle_handler)
step_pipeline_button.on_click(step_pipeline_handler)
run_all_button.on_click(run_all_handler)
reg_button.on_click(reg_button_handler); mem_button.on_click(mem_button_handler); cache_button.on_click(cache_button_handler)

# Inicializa as variáveis globais (serão preenchidas no 'Load/Reset')
regs, mem, cache = None, None, None

# Organiza os widgets na tela usando caixas horizontais (HBox) e verticais (VBox).
top_controls = widgets.HBox([load_button, step_single_cycle_button, step_pipeline_button, run_all_button, reg_button, mem_button, cache_button, start_addr, mem_range])
pipeline_options = widgets.VBox([hazard_mode_selector, cache_selector, cache_policy_selector, cache_penalty_input])
ui = widgets.VBox([part_selector, pipeline_options, code_area, top_controls, output_area])

# Chama a função de UI dinâmica uma vez para configurar o estado inicial correto.
on_part_selection_change({'new': part_selector.value})

print("Interface carregada. Execute os próximos blocos para definir a lógica do simulador.")

## Execução

In [ ]:
# Exibe a interface completa na célula do notebook.
display(ui)

# Componetes Comuns

In [ ]:
# ===================================================================
# Bloco 2: Componentes Comuns
# Define as classes de hardware (Memória, Registradores, Cache),
# funções auxiliares (conversão, formatação) e funções de exibição
# que são usadas por todo o simulador.
# ===================================================================

from IPython.display import display, HTML
import pandas as pd
import struct
import random # Necessário para a política de cache Random

# --- Classe de Cores para o Log ---
class LogColors:
    """Armazena códigos de escape ANSI para colorir a saída de texto."""
    INFO, SUCCESS, ERROR, WARNING, RESET = ('\033[94m', '\033[92m', '\033[91m', '\033[93m', '\033[0m')

# --- Classes do Simulador ---
class SimpleMemory:
    """Simula a memória principal (RAM) como uma lista de palavras de 32 bits."""
    def __init__(self, size_in_words=64):
        """Inicializa a memória com um tamanho específico, preenchida com zeros."""
        self.size = size_in_words
        self.mem = [0] * size_in_words

    def load_word(self, address):
        """Lê (carrega) uma palavra de 32 bits de um endereço específico."""
        if 0 <= address < self.size: return self.mem[address]
        raise IndexError(f"Endereço de memória inválido: {address}")

    def store_word(self, address, value):
        """Escreve (armazena) uma palavra de 32 bits em um endereço específico."""
        if 0 <= address < self.size: self.mem[address] = value & 0xFFFFFFFF # Garante 32 bits
        else: raise IndexError(f"Endereço de memória inválido: {address}")

class RegisterFile:
    """Simula o banco de 32 registradores da CPU (x0 a x31)."""
    def __init__(self):
        """Inicializa todos os 32 registradores com zero."""
        self.regs = [0] * 32

    def read(self, reg_num):
        """Lê o valor de um registrador específico."""
        if 0 <= reg_num < 32: return self.regs[reg_num]
        raise IndexError(f"Número de registrador inválido: x{reg_num}")

    def write(self, reg_num, value):
        """Escreve um valor em um registrador específico, ignorando x0."""
        if reg_num == 0: return # x0 é sempre zero
        if 0 < reg_num < 32: self.regs[reg_num] = value & 0xFFFFFFFF # Garante 32 bits
        else: raise IndexError(f"Número de registrador inválido: x{reg_num}")

# --- Funções Auxiliares ---
def float_to_int_bits(f):
    """Converte um float Python para sua representação em bits (inteiro de 32 bits)."""
    return struct.unpack('I', struct.pack('f', float(f)))[0]

def int_bits_to_float(i):
    """Converte uma representação em bits (inteiro de 32 bits) para um float Python."""
    try: return struct.unpack('f', struct.pack('I', i & 0xFFFFFFFF))[0]
    except: return float('nan') # Retorna NaN se os bits não formarem um float válido

def to_signed32(value):
    """Converte um inteiro Python para sua representação com sinal de 32 bits (complemento de dois)."""
    value &= 0xFFFFFFFF # Garante que temos apenas 32 bits
    # Verifica o bit de sinal (o 31º bit) e ajusta se for negativo
    return value - (1 << 32) if value & (1 << 31) else value

def format_instruction(instr):
    """Converte a representação interna de uma instrução (dicionário) para uma string Assembly legível."""
    if not instr: return "" # Retorna vazio se a instrução for None (ex: pipeline vazio)
    op = instr['op']
    # Formata a string baseado no tipo da instrução e seus operandos
    if op in ["ADD", "SUB", "MUL", "DIV", "ADD.S", "MUL.S"]: return f"{op:<8} x{instr['rd']}, x{instr['rs1']}, x{instr['rs2']}"
    if op == "ADDI": return f"{op:<8} x{instr['rd']}, x{instr['rs1']}, {instr['imm']}"
    if op == "LW": return f"{op:<8} x{instr['rd']}, {instr['addr']}"
    if op == "SW": return f"{op:<8} x{instr['rs1']}, {instr['addr']}"
    # Se for uma instrução de desvio, apenas mostra a operação (o target é interno)
    if op == "BEQ": return f"{op:<8} x{instr['rs1']}, x{instr['rs2']}, target"
    if op == "J": return f"{op:<8} target"
    # Fallback: se não reconhecer, mostra o dicionário
    return str(instr)

# --- Funções de Exibição ---
def show_registers(regs):
    """Exibe o conteúdo do banco de registradores em 4 colunas formatadas."""
    if not regs: # Verifica se o objeto regs existe
        with output_area: clear_output(wait=True); print(f"{LogColors.WARNING}Programa não carregado.{LogColors.RESET}")
        return
    output_widgets = []
    for i in range(0, 32, 8): # Itera em blocos de 8 registradores
        start_reg, end_reg = i, i + 8
        register_values = [regs.read(j) for j in range(start_reg, end_reg)]
        # Cria um dicionário com os dados formatados
        data = {"Reg": [f"x{j}" for j in range(start_reg, end_reg)],
                "Valor Int": [to_signed32(val) for val in register_values],
                "Valor FP": [f"{int_bits_to_float(val):.4f}" for val in register_values]}
        # Cria a tabela (DataFrame) e a coloca dentro de um widget Output
        df = pd.DataFrame(data).set_index("Reg")
        out = widgets.Output();
        with out: display(df)
        output_widgets.append(out)
    # Exibe as 4 tabelas (widgets Output) lado a lado usando um HBox
    with output_area: clear_output(wait=True); display(widgets.HBox(output_widgets))

def show_memory(mem, start=0, rng=10):
    """Exibe um trecho da memória principal em uma tabela."""
    if not mem: # Verifica se o objeto mem existe
        with output_area: clear_output(wait=True); print(f"{LogColors.WARNING}Programa não carregado.{LogColors.RESET}")
        return
    try:
        # Cria um dicionário com os endereços e os valores lidos da memória
        data = {"Endereço": list(range(start, start + rng)),
                "Valor": [mem.load_word(i) for i in range(start, start + rng)]}
        # Cria e exibe a tabela (DataFrame)
        df = pd.DataFrame(data).set_index("Endereço")
        with output_area: clear_output(wait=True); display(df)
    except IndexError as e: # Captura erros se o endereço for inválido
        with output_area: clear_output(wait=True); print(f"{LogColors.ERROR}Erro ao acessar memória: {e}{LogColors.RESET}")

def show_cache_stats(cache_obj):
    """Exibe as estatísticas de acertos e falhas da cache."""
    if not cache_obj: # Verifica se o objeto cache existe
        with output_area: clear_output(wait=True); print(f"{LogColors.WARNING}Programa não carregado.{LogColors.RESET}")
        return
    # Pega as estatísticas do objeto Cache
    hit_rate, miss_rate, hits, misses = cache_obj.get_stats()
    # Formata a string de saída com cores
    stats_str = (
        f"{LogColors.INFO}--- Estatísticas da Cache ---\n{LogColors.RESET}"
        f"Acessos Totais: {hits + misses}\n"
        f"Hits: {LogColors.SUCCESS}{hits}{LogColors.RESET}\n"
        f"Misses: {LogColors.ERROR}{misses}{LogColors.RESET}\n"
        f"Taxa de Acertos (Hit Rate): {LogColors.SUCCESS}{hit_rate:.2f}%{LogColors.RESET}\n"
        f"Taxa de Falhas (Miss Rate): {LogColors.ERROR}{miss_rate:.2f}%{LogColors.RESET}"
    )
    # Exibe as estatísticas formatadas
    with output_area: clear_output(wait=True); print(stats_str)

# --- Função para gerar o botão de "Copiar" (para o Bloco de Experimentos) ---
def create_copy_button(code_text, button_id):
    """Gera HTML/JS para um bloco de código com botão 'Copiar'."""
    area_id, btn_id = f"code-area-{button_id}", f"copy-button-{button_id}"
    html_content = f"""
    <div style="position: relative; border: 1px solid #ccc; border-radius: 5px; padding: 10px; margin-bottom: 15px; background-color: #2e2e2e; color: #f0f0f0;">
        <pre style="margin: 0; white-space: pre-wrap; word-wrap: break-word;"><code>{code_text}</code></pre>
        <button id="{btn_id}" onclick="copyToClipboard('{area_id}', '{btn_id}')" style="position: absolute; top: 10px; right: 10px; padding: 5px 10px; border: 1px solid #555; border-radius: 5px; cursor: pointer; background-color: #444; color: #fff;">
            Copiar
        </button>
        <textarea id="{area_id}" style="position: absolute; left: -9999px; top: 0;">{code_text}</textarea>
    </div>
    <script>
    if (typeof copyToClipboard === 'undefined') {{ // Define a função JS apenas uma vez
        function copyToClipboard(elementId, buttonId) {{
            const codeElement = document.getElementById(elementId);
            codeElement.select(); codeElement.setSelectionRange(0, 99999); // Seleciona o texto
            navigator.clipboard.writeText(codeElement.value).then(() => {{ // Copia para o clipboard
                const button = document.getElementById(buttonId);
                button.innerText = 'Copiado!'; button.style.backgroundColor = '#3a8a3a'; // Feedback visual
                setTimeout(() => {{ button.innerText = 'Copiar'; button.style.backgroundColor = '#444'; }}, 2000); // Volta ao normal
            }}).catch(err => console.error('Falha ao copiar texto: ', err));
        }}
    }}
    </script>"""
    return HTML(html_content)

print(f"{LogColors.SUCCESS}Componentes comuns carregados.{LogColors.RESET}")

# Parte 2

In [ ]:
# ===================================================================
# Bloco 3: Lógica da Parte 2 (Simulador de Ciclo Único - Base)
# Define a versão inicial e simplificada do parser e do executor,
# contendo apenas as instruções requeridas pela Parte 2 (ADD, SUB,
# MUL, LW, SW). Estas funções serão sobrescritas pelo Bloco 4.
# ===================================================================

def execute_instruction_single_cycle(instr, regs, mem, pc=0): # Adicionado pc para compatibilidade
    """
    Versão base do executor de instruções.
    Recebe uma instrução (dicionário) e modifica o estado (regs, mem).
    Só entende ADD, SUB, MUL, LW, SW.
    Retorna o próximo PC (sempre pc+1 nesta versão).
    """
    op = instr["op"]
    if op in ["ADD", "SUB", "MUL"]:
        rs1_val, rs2_val = to_signed32(regs.read(instr["rs1"])), to_signed32(regs.read(instr["rs2"]))
        if op == "ADD": result = rs1_val + rs2_val
        elif op == "SUB": result = rs1_val - rs2_val
        else: result = rs1_val * rs2_val
        regs.write(instr["rd"], result)
    elif op == "LW": regs.write(instr["rd"], mem.load_word(instr["addr"]))
    elif op == "SW": mem.store_word(instr["addr"], regs.read(instr["rs1"]))
    else: raise ValueError(f"Instrução '{op}' não pertence ao conjunto base da Parte 2.")
    return pc + 1 # Em ciclo único (sem desvios), o próximo PC é sempre o atual + 1

def parse_program_single_cycle(asm_code):
    """
    Versão base do parser.
    Converte código Assembly (string) em uma lista de instruções (dicionários).
    Só reconhece ADD, SUB, MUL, LW, SW.
    """
    program = []
    # Limpa linhas vazias e comentários
    cleaned_lines = [line.strip().split('#')[0].strip() for line in asm_code.strip().splitlines()]
    for line in cleaned_lines:
        if not line: continue
        parts = line.replace(",", "").split()
        op = parts[0].upper()
        # Mapeia a string da instrução para um dicionário estruturado
        if op in ["ADD", "SUB", "MUL"]: instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "rs2": int(parts[3][1:])}
        elif op == "LW": instr = {"op": op, "rd": int(parts[1][1:]), "addr": int(parts[2])}
        elif op == "SW": instr = {"op": op, "rs1": int(parts[1][1:]), "addr": int(parts[2])}
        else: raise ValueError(f"Instrução '{op}' não reconhecida pelo parser base.")
        program.append(instr)
    return program

print(f"{LogColors.SUCCESS}Funções base de simulação (Parte 2) carregadas.{LogColors.RESET}")

# Parte 1


In [ ]:
# ===================================================================
# Bloco 4: Lógica da Parte 1 (Extensões para o Simulador)
#
# SOBRESCREVE as funções parse_program_single_cycle e
# execute_instruction_single_cycle do Bloco 3, adicionando
# suporte para DIV, ADDI, Ponto Flutuante (ADD.S, MUL.S) e
# detecção de overflow.
# ===================================================================

def execute_instruction_single_cycle(instr, regs, mem, pc): # Assinatura atualizada
    """
    Versão completa do executor (sobrescreve a do Bloco 3).
    Executa instruções das Partes 1 e 2, incluindo DIV, ADDI, floats,
    overflow e desvios (BEQ, J).
    Retorna o endereço da PRÓXIMA instrução a ser executada.
    """
    op = instr["op"]; INT32_MIN, INT32_MAX = -2**31, 2**31 - 1
    next_pc = pc + 1 # Próximo PC padrão

    # Lógica para instruções aritméticas de inteiros
    if op in ["ADD", "ADDI", "SUB", "MUL", "DIV"]:
        rs1_val = to_signed32(regs.read(instr["rs1"]));
        rs2_val = instr["imm"] if op == "ADDI" else to_signed32(regs.read(instr["rs2"]))
        if op in ["ADD", "ADDI"]: result = rs1_val + rs2_val
        elif op == "SUB": result = rs1_val - rs2_val
        elif op == "MUL": result = rs1_val * rs2_val
        elif op == "DIV":
            if rs2_val == 0: raise ValueError("Divisão por zero")
            if rs1_val == INT32_MIN and rs2_val == -1: raise OverflowError("Overflow na divisão")
            result = int(rs1_val / rs2_val)
        # Verifica overflow (exceto para DIV, que já foi tratada)
        if not (INT32_MIN <= result <= INT32_MAX) and op != "DIV": raise OverflowError(f"Overflow na operação")
        regs.write(instr["rd"], result)

    # Lógica para instruções de ponto flutuante
    elif op in ["ADD.S", "MUL.S"]:
        float1, float2 = int_bits_to_float(regs.read(instr["rs1"])), int_bits_to_float(regs.read(instr["rs2"]))
        result_float = (float1 + float2) if op == "ADD.S" else (float1 * float2)
        regs.write(instr["rd"], float_to_int_bits(result_float))

    # Lógica para instruções de memória
    elif op == "LW": regs.write(instr["rd"], mem.load_word(instr["addr"]))
    elif op == "SW": mem.store_word(instr["addr"], regs.read(instr["rs1"]))

    # Lógica para instruções de desvio (necessário para o parser unificado)
    elif op == "BEQ":
        if regs.read(instr['rs1']) == regs.read(instr['rs2']):
            next_pc = instr['target'] # Atualiza o próximo PC se a condição for verdadeira
    elif op == "J":
        next_pc = instr['target'] # Atualiza o próximo PC incondicionalmente

    else: raise ValueError(f"Instrução desconhecida {op}")

    return next_pc # Retorna o endereço da próxima instrução

def parse_program_single_cycle(asm_code):
    """
    Versão completa do parser (sobrescreve a do Bloco 3).
    Reconhece todas as instruções das Partes 1 e 2 (incluindo DIV, ADDI, floats).
    NOTA: Esta função ainda NÃO reconhece labels ou desvios. O parser
    final (Bloco 9) é usado para isso no modo Pipeline. Para Ciclo Único,
    usaremos este parser, significando que desvios não funcionarão
    neste modo específico via Bloco 1 (mas a lógica de execução existe).
    Se quisermos desvios no ciclo único, precisaríamos usar parse_program_final
    no Bloco 1 quando p1_2 for selecionado.
    """
    program = []
    cleaned_lines = [line.strip().split('#')[0].strip() for line in asm_code.strip().splitlines()]
    for line in cleaned_lines:
        if not line: continue
        parts = line.replace(",", "").split()
        op = parts[0].upper()
        if op in ["ADD", "SUB", "MUL", "DIV", "ADD.S", "MUL.S"]: instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "rs2": int(parts[3][1:])}
        elif op == "ADDI": instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "imm": int(parts[3])}
        elif op == "LW": instr = {"op": op, "rd": int(parts[1][1:]), "addr": int(parts[2])}
        elif op == "SW": instr = {"op": op, "rs1": int(parts[1][1:]), "addr": int(parts[2])}
        else: raise ValueError(f"Instrução desconhecida: {line}")
        program.append(instr)
    return program

print(f"{LogColors.SUCCESS}Funções de simulação da Parte 1 e 2 (versão completa) carregadas.{LogColors.RESET}")

# Parte 3

In [ ]:
# ===================================================================
# Bloco 5: Lógica da Parte 3 (Simulador de Pipeline - Base)
# Define a primeira versão da função run_program_pipeline, que
# apenas simula o movimento das instruções pelos estágios, sem
# executar a lógica interna nem tratar hazards. Será sobrescrita.
# ===================================================================

def run_program_pipeline(asm_code, regs, mem, cache, hazard_mode='none', cache_mode='disabled', miss_penalty=1):
    """
    Versão base da simulação de pipeline (sobrescrita depois).
    Apenas move as instruções pelos estágios e gera a tabela.
    """
    try: program = parse_program_final(asm_code) # Usa o parser final que entende desvios
    except (ValueError, KeyError) as e: print(f"{LogColors.ERROR}ERRO DE SINTAXE: {e}{LogColors.RESET}"); return

    pc, cycle = 0, 0
    pipeline_stages = {'IF': None, 'ID': None, 'EX': None, 'MEM_WB': None}
    history = []

    while pc < len(program) or any(s is not None for s in pipeline_stages.values()):
        cycle += 1
        prev_stages = pipeline_stages.copy()

        # Simplesmente move as instruções para o próximo estágio
        pipeline_stages['MEM_WB'] = prev_stages['EX']
        pipeline_stages['EX'] = prev_stages['ID']
        pipeline_stages['ID'] = prev_stages['IF']

        # Busca a próxima instrução
        if pc < len(program):
            pipeline_stages['IF'] = program[pc]
            pc += 1
        else:
            pipeline_stages['IF'] = None

        # Registra o estado do pipeline neste ciclo
        history.append({ 'Ciclo': cycle,
            'IF': format_instruction(pipeline_stages['IF']), 'ID': format_instruction(pipeline_stages['ID']),
            'EX': format_instruction(pipeline_stages['EX']), 'MEM_WB': format_instruction(pipeline_stages['MEM_WB']) })

        # Limite de segurança para evitar loops infinitos
        if cycle > len(program) + 10: print(f"{LogColors.WARNING}Simulação parada: limite de ciclos.{LogColors.RESET}"); break

    print(f"\n{LogColors.INFO}--- Tabela de Execução do Pipeline ---{LogColors.RESET}")
    if history:
        df = pd.DataFrame(history).set_index('Ciclo')
        with output_area: clear_output(wait=True); display(df.fillna(""))

    print(f"\n{LogColors.INFO}Total de Ciclos (Pipeline):{LogColors.RESET} {cycle}\n{LogColors.INFO}Total de Ciclos (Ciclo Único seria):{LogColors.RESET} {len(program)}")

print(f"{LogColors.SUCCESS}Função base de simulação de Pipeline (Parte 3) carregada.{LogColors.RESET}")

# Parte 4

In [ ]:
# ===================================================================
# Bloco 6: Lógica da Parte 4 (Pipeline com Data Hazards)
#
# SOBRESCREVE a função run_program_pipeline do Bloco 5.
# Adiciona a execução real das instruções nos estágios EX e MEM_WB.
# Implementa a detecção de Data Hazards (EX->ID e Load-Use) e
# as lógicas de tratamento por Stall e Forwarding.
# ===================================================================

def run_program_pipeline(asm_code, regs, mem, cache, hazard_mode='forwarding', cache_mode='disabled', miss_penalty=1):
    """
    Versão aprimorada do pipeline (sobrescreve a do Bloco 5).
    Executa instruções e trata Data Hazards.
    """
    try: program = parse_program_final(asm_code)
    except (ValueError, KeyError) as e: print(f"{LogColors.ERROR}ERRO DE SINTAXE: {e}{LogColors.RESET}"); return

    pc, cycle, stalls = 0, 0, 0
    pipeline_stages = {'IF': None, 'ID': None, 'EX': None, 'MEM_WB': None}
    history, hazard_logs = [], []

    while pc < len(program) or any(s is not None for s in pipeline_stages.values()):
        cycle += 1
        prev_stages = deepcopy(pipeline_stages) # Cópia profunda para evitar efeitos colaterais
        stall_this_cycle = False

        # --- Execução dos Estágios (ordem reversa) ---

        # ESTÁGIO 4: MEMORY ACCESS & WRITE BACK (MEM_WB)
        instr_wb = prev_stages['MEM_WB']
        if instr_wb:
            op = instr_wb['op']
            # Escreve o resultado da ALU ou Load no banco de registradores
            if 'result' in instr_wb and op != 'SW': # SW não escreve em registrador
                 regs.write(instr_wb['rd'], instr_wb['result'])
            # Escreve na memória para SW (acesso direto, sem cache ainda)
            elif op == 'SW':
                 mem.store_word(instr_wb['addr'], regs.read(instr_wb['rs1']))

        # ESTÁGIO 3: EXECUTE (EX)
        instr_ex = prev_stages['EX']
        if instr_ex:
            op = instr_ex['op']
            # Pega os operandos (do banco de registradores)
            val1 = regs.read(instr_ex.get('rs1', 0))
            val2 = regs.read(instr_ex.get('rs2', 0)) if 'rs2' in instr_ex else instr_ex.get('imm', 0)

            # --- LÓGICA DE FORWARDING (MEM -> EX) ---
            if hazard_mode == 'forwarding':
                instr_mem = prev_stages['MEM_WB'] # Instrução que ESTAVA em MEM no início do ciclo
                # Se a instrução em MEM_WB calculou um resultado e seu destino é uma das fontes de EX...
                if instr_mem and 'result' in instr_mem and 'rd' in instr_mem and instr_mem['op'] != 'SW':
                    fwd_reg = instr_mem['rd']
                    if fwd_reg == instr_ex.get('rs1'):
                        val1 = instr_mem['result'] # Pega o valor adiantado!
                        hazard_logs.append(f"Ciclo {cycle}: 🟢 FWD(MEM->EX) x{fwd_reg} p/ '{format_instruction(instr_ex)}'")
                    if 'rs2' in instr_ex and fwd_reg == instr_ex.get('rs2'):
                        val2 = instr_mem['result'] # Pega o valor adiantado!
                        hazard_logs.append(f"Ciclo {cycle}: 🟢 FWD(MEM->EX) x{fwd_reg} p/ '{format_instruction(instr_ex)}'")

            # Execução da ALU ou Leitura da Memória
            if op in ["ADD", "ADDI"]: instr_ex['result'] = to_signed32(val1) + to_signed32(val2)
            elif op == "SUB": instr_ex['result'] = to_signed32(val1) - to_signed32(val2)
            elif op == "LW": instr_ex['result'] = mem.load_word(instr_ex['addr']) # LW lê memória aqui
            # Resultado é anexado para o próximo estágio
            pipeline_stages['MEM_WB'] = instr_ex
        else:
            pipeline_stages['MEM_WB'] = None # Propaga a bolha

        # ESTÁGIO 2: INSTRUCTION DECODE (ID) & HAZARD DETECTION
        instr_id = prev_stages['ID']
        if instr_id:
            src_regs = {instr_id.get('rs1'), instr_id.get('rs2')} - {None} # Registradores que ID quer ler
            instr_ex_prev = prev_stages['EX'] # Instrução que ESTAVA em EX

            # Verifica se há dependência EX -> ID
            if instr_ex_prev and 'rd' in instr_ex_prev and instr_ex_prev['rd'] != 0 and instr_ex_prev['rd'] in src_regs:
                hazard_reg = instr_ex_prev['rd']
                # Caso especial: Hazard de Load-Use
                if instr_ex_prev['op'] == 'LW':
                    if hazard_mode != 'none': # Stall é obrigatório mesmo com forwarding
                        stall_this_cycle = True
                        hazard_logs.append(f"Ciclo {cycle}: 🔴 HAZARD DE LOAD-USE x{hazard_reg}. Ação: STALL.")
                    else: hazard_logs.append(f"Ciclo {cycle}: 🔴 HAZARD DE LOAD-USE x{hazard_reg}. 🟡 Ação: Nenhuma.")
                # Caso normal: Hazard de Dados
                else:
                    if hazard_mode == 'stall':
                        stall_this_cycle = True
                        hazard_logs.append(f"Ciclo {cycle}: 🟡 HAZARD DE DADOS x{hazard_reg}. Ação: STALL.")
                    elif hazard_mode == 'none':
                        hazard_logs.append(f"Ciclo {cycle}: 🔴 HAZARD DE DADOS x{hazard_reg}. 🟡 Ação: Nenhuma.")
                        # Forwarding é tratado no estágio EX, não causa stall aqui.

        # --- AVANÇO DO PIPELINE (Controlado por Stall) ---
        if stall_this_cycle:
            stalls += 1
            pipeline_stages['EX'] = None # Insere bolha em EX
            # IF e ID permanecem os mesmos (pc não avança)
            pipeline_stages['ID'] = prev_stages['ID']
            pipeline_stages['IF'] = prev_stages['IF']
        else:
            # Avanço normal
            pipeline_stages['EX'] = prev_stages['ID']
            pipeline_stages['ID'] = prev_stages['IF']
            # ESTÁGIO 1: INSTRUCTION FETCH (IF)
            if pc < len(program):
                pipeline_stages['IF'] = program[pc]
                pc += 1
            else:
                pipeline_stages['IF'] = None

        # Registra o estado do pipeline neste ciclo
        history.append({ 'Ciclo': cycle,
            'IF': format_instruction(pipeline_stages['IF']), 'ID': format_instruction(pipeline_stages['ID']),
            'EX': f"*{format_instruction(pipeline_stages['EX'])}" if stall_this_cycle else format_instruction(pipeline_stages['EX']),
            'MEM_WB': format_instruction(pipeline_stages['MEM_WB'])})

        # Limite de segurança
        if cycle > (len(program) + stalls) * 2 and cycle > 30:
            print(f"{LogColors.WARNING}Simulação parada: limite de ciclos atingido.{LogColors.RESET}"); break

    # --- Finalização ---
    if hazard_logs: print(f"\n{LogColors.INFO}--- Detalhes de Hazards e Forwarding ---{LogColors.RESET}"); [print(log) for log in hazard_logs]
    print(f"\n{LogColors.INFO}--- Tabela de Execução do Pipeline ---{LogColors.RESET}")
    if history:
        df = pd.DataFrame(history).set_index('Ciclo')
        with output_area: clear_output(wait=True); display(df.fillna(""))
    print(f"\n{LogColors.INFO}Total de Ciclos (Pipeline):{LogColors.RESET} {cycle}\n{LogColors.INFO}Número de Stalls (bolhas):{LogColors.RESET} {stalls}\n{LogColors.INFO}Total de Ciclos (Ciclo Único seria):{LogColors.RESET} {len(program)}")

print(f"{LogColors.SUCCESS}Função de simulação de Pipeline com Hazards (Parte 4) carregada.{LogColors.RESET}")

# Parte 5

In [ ]:
# ===================================================================
# Bloco 7: Lógica da Parte 5 (Hierarquia de Memória - Cache)
# Define a classe da Cache com suporte a políticas LRU e Random,
# e métodos para ler, escrever e obter estatísticas.
# ===================================================================
import random

class Cache:
    """Simula uma cache associativa por conjunto de 2 vias."""
    def __init__(self, num_sets, main_memory, replacement_policy='lru'):
        """Inicializa a cache com N conjuntos, referência à memória principal e política."""
        self.num_sets = num_sets
        self.main_memory = main_memory
        self.replacement_policy = replacement_policy
        # Cria a estrutura: lista de conjuntos, onde cada conjunto é uma lista de 2 linhas (dicionários)
        self.sets = [[self._create_cache_line(), self._create_cache_line()] for _ in range(num_sets)]
        self.hits, self.misses = 0, 0 # Contadores de estatísticas

    def _create_cache_line(self):
        """Retorna um dicionário representando uma linha de cache vazia."""
        return {'valid': 0, 'tag': 0, 'data': 0, 'lru': 0} # lru=0 -> Via 0 usada por último, lru=1 -> Via 1 usada por último

    def _get_address_parts(self, address):
        """Calcula o índice do conjunto e a tag a partir do endereço."""
        index = address % self.num_sets
        tag = address // self.num_sets
        return index, tag

    def read(self, address):
        """
        Tenta ler um dado da cache a partir de um endereço.
        Retorna uma tupla (dado, status), onde status é 'hit' ou 'miss'.
        """
        index, tag = self._get_address_parts(address)
        target_set = self.sets[index]

        # 1. Procura por um HIT nas duas vias do conjunto
        for i, line in enumerate(target_set):
            if line['valid'] == 1 and line['tag'] == tag:
                self.hits += 1
                # Atualiza LRU: marca a outra via como menos recente
                target_set[0]['lru'], target_set[1]['lru'] = (1-i), i
                return line['data'], 'hit'

        # 2. Se não encontrou, é um MISS
        self.misses += 1
        data = self.main_memory.load_word(address) # Busca da memória principal

        # 3. Escolhe qual via substituir
        if self.replacement_policy == 'lru':
            # Encontra a via cujo bit LRU NÃO corresponde ao seu próprio índice (i)
            way_to_replace = next((i for i, line in enumerate(target_set) if line['lru'] != i), 0)
        else: # random
            way_to_replace = random.choice([0, 1])

        # 4. Atualiza a linha da cache com os novos dados
        target_set[way_to_replace].update({'valid': 1, 'tag': tag, 'data': data})
        # Atualiza LRU: marca a via que acabou de ser usada como a mais recente
        target_set[0]['lru'], target_set[1]['lru'] = (1-way_to_replace), way_to_replace

        return data, 'miss'

    def write(self, address, data):
        """
        Escreve um dado na memória/cache (política Write-Through, No-Write-Allocate).
        Retorna uma tupla (None, status), onde status é 'hit' ou 'miss'.
        """
        index, tag = self._get_address_parts(address)
        target_set = self.sets[index]
        self.main_memory.store_word(address, data) # Write-Through: Sempre escreve na memória

        # Procura por um HIT para atualizar a cache também
        for i, line in enumerate(target_set):
            if line['valid'] == 1 and line['tag'] == tag: # Write-Hit
                self.hits += 1
                line['data'] = data # Atualiza o dado na cache
                target_set[0]['lru'], target_set[1]['lru'] = (1-i), i # Atualiza LRU
                return None, 'hit'

        # Write-Miss + No-Write-Allocate: Não faz nada na cache
        self.misses += 1
        return None, 'miss'

    def get_stats(self):
        """Calcula e retorna as estatísticas da cache."""
        total = self.hits + self.misses
        if total == 0: return 0.0, 0.0, 0, 0
        hit_rate = (self.hits / total) * 100
        miss_rate = (self.misses / total) * 100
        return hit_rate, miss_rate, self.hits, self.misses

print(f"{LogColors.SUCCESS}Componente de Cache com Políticas de Substituição (Parte 5) carregado.{LogColors.RESET}")

In [ ]:
# ===================================================================
# Bloco 8: Integração Final (Pipeline + Cache)
#
# SOBRESCREVE a função run_program_pipeline do Bloco 6.
# Modifica os estágios MEM_WB e EX para interagir com a Cache
# (usando cache.read/write) em vez da memória principal,
# baseado na seleção do usuário (cache_mode).
# ===================================================================

def run_program_pipeline(asm_code, regs, mem, cache, hazard_mode='forwarding', cache_mode='enabled', miss_penalty=1):
    """
    Versão aprimorada do pipeline (sobrescreve Bloco 6).
    Integra a Cache nos acessos de memória (LW/SW).
    """
    try: program = parse_program_final(asm_code)
    except (ValueError, KeyError) as e: print(f"{LogColors.ERROR}ERRO DE SINTAXE: {e}{LogColors.RESET}"); return

    pc, cycle, stalls, mem_stalls = 0, 0, 0, 0
    pipeline_stages = {'IF': None, 'ID': None, 'EX': None, 'MEM_WB': None}
    history, hazard_logs = [], []

    while pc < len(program) or any(s is not None for s in pipeline_stages.values()):
        cycle += 1

        # --- Simulação de Penalidade de Cache Miss ---
        if mem_stalls > 0:
            mem_stalls -= 1
            hazard_logs.append(f"Ciclo {cycle}: 🟡 STALL DE MEMÓRIA (Cache Miss)")
            # Adiciona uma linha de stall ao histórico
            history.append({ 'Ciclo': cycle, 'IF': "STALL", 'ID': "STALL", 'EX': "STALL", 'MEM_WB': format_instruction(pipeline_stages['MEM_WB'])})
            continue # Pula o resto do ciclo, efetivamente parando o pipeline

        prev_stages = deepcopy(pipeline_stages)
        stall_this_cycle = False

        # --- Execução dos Estágios (ordem reversa) ---

        # ESTÁGIO 4: MEMORY ACCESS & WRITE BACK (MEM_WB)
        instr_wb = prev_stages['MEM_WB']
        if instr_wb:
            op = instr_wb['op']

            # --- INTEGRAÇÃO COM A CACHE ---
            if op == "LW":
                address = instr_wb['addr']
                # Decide se usa a cache ou a memória principal
                if cache_mode == 'enabled':
                    data, status = cache.read(address)
                    if status == 'miss' and miss_penalty > 1: mem_stalls = miss_penalty - 1 # Agenda stalls para os próximos ciclos
                else: # Acesso direto à memória
                    data, status = mem.load_word(address), 'hit' # Assume hit (sem penalidade)
                instr_wb['result'] = data # Anexa o dado lido (para possível forwarding)
                regs.write(instr_wb['rd'], data) # Escreve no registrador

            elif op == "SW":
                address = instr_wb['addr']
                data_to_store = regs.read(instr_wb['rs1'])
                # Decide se usa a cache ou a memória principal
                if cache_mode == 'enabled':
                    _, status = cache.write(address, data_to_store)
                    if status == 'miss' and miss_penalty > 1: mem_stalls = miss_penalty - 1 # Agenda stalls
                else: # Acesso direto à memória
                    mem.store_word(address, data_to_store)
                    status = 'hit'

            elif 'result' in instr_wb: # Instruções Aritméticas (apenas escreve no reg)
                regs.write(instr_wb['rd'], instr_wb['result'])
            # --- FIM DA INTEGRAÇÃO ---

        # ESTÁGIO 3: EXECUTE (EX)
        instr_ex = prev_stages['EX']
        if instr_ex:
            op = instr_ex['op']
            val1 = regs.read(instr_ex.get('rs1', 0))
            val2 = regs.read(instr_ex.get('rs2', 0)) if 'rs2' in instr_ex else instr_ex.get('imm', 0)

            if hazard_mode == 'forwarding':
                instr_mem = prev_stages['MEM_WB'] # Instrução que ESTAVA em MEM
                # Forwarding MEM -> EX (apenas se não for SW e tiver resultado)
                if instr_mem and 'result' in instr_mem and 'rd' in instr_mem and instr_mem['op'] != 'SW':
                    fwd_reg = instr_mem['rd']
                    if fwd_reg != 0 and fwd_reg == instr_ex.get('rs1'):
                        val1 = instr_mem['result']; hazard_logs.append(f"Ciclo {cycle}: 🟢 FWD(MEM->EX) x{fwd_reg} p/ '{format_instruction(instr_ex)}'")
                    if 'rs2' in instr_ex and fwd_reg != 0 and fwd_reg == instr_ex.get('rs2'):
                        val2 = instr_mem['result']; hazard_logs.append(f"Ciclo {cycle}: 🟢 FWD(MEM->EX) x{fwd_reg} p/ '{format_instruction(instr_ex)}'")

            # Execução da ALU
            if op in ["ADD", "ADDI"]: instr_ex['result'] = to_signed32(val1) + to_signed32(val2)
            elif op == "SUB": instr_ex['result'] = to_signed32(val1) - to_signed32(val2)
            # Para LW/SW, o estágio EX apenas passa a instrução adiante nesta simulação simplificada
            # (em um processador real, calcularia o endereço aqui).
            pipeline_stages['MEM_WB'] = instr_ex
        else:
            pipeline_stages['MEM_WB'] = None

        # ESTÁGIO 2: INSTRUCTION DECODE (ID) & HAZARD DETECTION
        instr_id = prev_stages['ID']
        if instr_id:
            src_regs = {instr_id.get('rs1'), instr_id.get('rs2')} - {None, 0} # Ignora x0 como fonte de hazard
            instr_ex_prev = prev_stages['EX'] # Instrução que ESTAVA em EX

            if instr_ex_prev and 'rd' in instr_ex_prev and instr_ex_prev['rd'] != 0 and instr_ex_prev['rd'] in src_regs:
                hazard_reg = instr_ex_prev['rd']
                if instr_ex_prev['op'] == 'LW': # Hazard de Load-Use
                    if hazard_mode != 'none': stall_this_cycle = True; hazard_logs.append(f"Ciclo {cycle}: 🔴 HAZARD DE LOAD-USE x{hazard_reg}. Ação: STALL.")
                    else: hazard_logs.append(f"Ciclo {cycle}: 🔴 HAZARD DE LOAD-USE x{hazard_reg}. 🟡 Ação: Nenhuma.")
                else: # Hazard de Dados normal
                    if hazard_mode == 'stall': stall_this_cycle = True; hazard_logs.append(f"Ciclo {cycle}: 🟡 HAZARD DE DADOS x{hazard_reg}. Ação: STALL.")
                    elif hazard_mode == 'none': hazard_logs.append(f"Ciclo {cycle}: 🔴 HAZARD DE DADOS x{hazard_reg}. 🟡 Ação: Nenhuma.")

        # --- AVANÇO DO PIPELINE ---
        if stall_this_cycle:
            stalls += 1; pipeline_stages['EX'] = None # Insere bolha
            pipeline_stages['ID'] = prev_stages['ID'] # Congela ID
            pipeline_stages['IF'] = prev_stages['IF'] # Congela IF (e PC)
        else:
            pipeline_stages['EX'] = prev_stages['ID']
            pipeline_stages['ID'] = prev_stages['IF']
            # ESTÁGIO 1: INSTRUCTION FETCH (IF)
            if pc < len(program): pipeline_stages['IF'] = program[pc]; pc += 1
            else: pipeline_stages['IF'] = None

        history.append({ 'Ciclo': cycle,
            'IF': format_instruction(pipeline_stages['IF']), 'ID': format_instruction(pipeline_stages['ID']),
            'EX': f"*{format_instruction(pipeline_stages['EX'])}" if stall_this_cycle else format_instruction(pipeline_stages['EX']),
            'MEM_WB': format_instruction(pipeline_stages['MEM_WB'])})
        if cycle > (len(program) + stalls + (miss_penalty * cache.misses)) * 1.5 and cycle > 50: # Limite de segurança dinâmico
             hazard_logs.append("Simulação parada: limite de ciclos atingido."); break

    # --- Finalização ---
    if hazard_logs: print(f"\n{LogColors.INFO}--- Detalhes de Hazards e Stalls ---{LogColors.RESET}"); [print(log) for log in hazard_logs]
    print(f"\n{LogColors.INFO}--- Tabela de Execução do Pipeline ---{LogColors.RESET}")
    if history:
        df = pd.DataFrame(history).set_index('Ciclo')
        with output_area: clear_output(wait=True); display(df.fillna(""))
    print(f"\n{LogColors.INFO}Total de Ciclos (Pipeline):{LogColors.RESET} {cycle}\n{LogColors.INFO}Número de Stalls (Data Hazards):{LogColors.RESET} {stalls}\n{LogColors.INFO}Total de Ciclos (Ciclo Único seria):{LogColors.RESET} {len(program)}")
    speedup = len(program) / cycle if cycle > 0 else 0
    print(f"{LogColors.SUCCESS}Speedup (vs Ciclo Único): {speedup:.2f}x{LogColors.RESET}")


print(f"{LogColors.SUCCESS}Função de simulação de Pipeline com Cache (Parte 5 - FINAL) carregada.{LogColors.RESET}")

# Parte Extra

In [ ]:
# ===================================================================
# Bloco 9: Parte Extra (Lógica de Execução Final e Step-by-Step)
# Define o parser final (com labels/desvios) e as funções de
# inicialização e execução passo a passo para ambos os modos
# (Ciclo Único e Pipeline).
# ===================================================================

from copy import deepcopy

# --- Parser Final (com labels e desvios) ---
def parse_program_final(asm_code):
    """Parser unificado que entende todas as instruções, incluindo labels e desvios."""
    initial_lines = [line.strip().split('#')[0].strip() for line in asm_code.strip().splitlines()]
    lines = [line for line in initial_lines if line] # Filtra linhas vazias
    labels, program_lines, current_addr = {}, [], 0
    # Primeira passada: encontra labels e separa as linhas de instrução
    for line in lines:
        if ':' in line:
            label, rest = line.split(':', 1)
            labels[label.strip()] = current_addr
            if rest.strip(): # Se houver instrução na mesma linha do label
                program_lines.append(rest.strip())
                current_addr += 1
        else:
            program_lines.append(line)
            current_addr += 1
    # Segunda passada: converte as linhas de instrução em dicionários
    program = []
    for addr, line in enumerate(program_lines):
        parts = line.replace(",", "").split()
        op = parts[0].upper()
        try:
            if op in ["ADD", "SUB", "MUL", "DIV", "ADD.S", "MUL.S"]: instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "rs2": int(parts[3][1:])}
            elif op == "ADDI": instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "imm": int(parts[3])}
            elif op == "LW": instr = {"op": op, "rd": int(parts[1][1:]), "addr": int(parts[2])}
            elif op == "SW": instr = {"op": op, "rs1": int(parts[1][1:]), "addr": int(parts[2])}
            elif op == "BEQ": instr = {"op": op, "rs1": int(parts[1][1:]), "rs2": int(parts[2][1:]), "target": labels[parts[3]]}
            elif op == "J": instr = {"op": op, "target": labels[parts[1]]}
            else: raise ValueError(f"Instrução desconhecida: {line}")
            program.append(instr)
        except (IndexError, KeyError, ValueError) as e:
             raise ValueError(f"Erro ao parsear linha '{line}': {e}")
    return program

# --- Funções de Simulação para Ciclo Único ---
def initialize_single_cycle_simulation():
    """Reseta o estado para uma nova simulação de Ciclo Único."""
    global sc_program, sc_pc, regs, mem
    regs = RegisterFile(); mem = SimpleMemory(64)
    try:
        # Usa o parser final para permitir desvios no ciclo único também
        sc_program = parse_program_final(code_area.value)
        sc_pc = 0 # Program Counter para ciclo único
        print(f"{LogColors.SUCCESS}Programa carregado com {len(sc_program)} instruções. Pronto para iniciar.{LogColors.RESET}")
        show_registers(regs) # Mostra estado inicial
    except (ValueError, KeyError) as e:
        print(f"{LogColors.ERROR}ERRO DE SINTAXE AO CARREGAR: {e}{LogColors.RESET}"); sc_program, sc_pc = [], 0

def execute_one_single_cycle_step():
    """Executa uma única instrução no modo Ciclo Único e atualiza o PC."""
    global sc_pc, sc_program, regs, mem
    # Verifica se o programa está carregado e se ainda há instruções
    if not sc_program or not (0 <= sc_pc < len(sc_program)):
        if sc_program: print(f"{LogColors.SUCCESS}Fim do programa.{LogColors.RESET}")
        else: print(f"{LogColors.WARNING}Nenhum programa carregado.{LogColors.RESET}")
        return True # Sinaliza que a execução terminou

    instr = sc_program[sc_pc]
    formatted_instr = format_instruction(instr)
    current_pc = sc_pc # Guarda o PC atual para o log
    try:
        # Chama o executor, que retorna o PRÓXIMO PC
        next_pc = execute_instruction_single_cycle(instr, regs, mem, current_pc)
        print(f"{LogColors.INFO}Passo {current_pc+1}:{LogColors.RESET} {formatted_instr:<25} -> {LogColors.SUCCESS}Sucesso!{LogColors.RESET}")
        sc_pc = next_pc # Atualiza o PC global
    except (ValueError, IndexError, OverflowError) as e:
        print(f"{LogColors.INFO}Passo {current_pc+1}:{LogColors.RESET} {formatted_instr:<25} -> {LogColors.ERROR}ERRO: {e}{LogColors.RESET}")
        sc_pc += 1 # Em caso de erro, apenas avança para a próxima linha

    show_registers(regs) # Atualiza a exibição
    return sc_pc >= len(sc_program) # Retorna True se a próxima instrução estiver fora dos limites

# --- Funções de Simulação para Pipeline ---
def initialize_pipeline_simulation():
    """Reseta o estado para uma nova simulação de Pipeline."""
    global p_program, p_pc, p_cycle, p_stalls, p_mem_stalls, p_pipeline_stages, p_history, p_hazard_logs, regs, mem, cache
    regs = RegisterFile(); mem = SimpleMemory(64)
    # Cria a cache com a política selecionada na UI
    cache = Cache(num_sets=8, main_memory=mem, replacement_policy=cache_policy_selector.value)
    try:
        p_program = parse_program_final(code_area.value) # Usa o parser final
        p_pc, p_cycle, p_stalls, p_mem_stalls = 0, 0, 0, 0
        p_pipeline_stages = {'IF': None, 'ID': None, 'EX': None, 'MEM_WB': None}
        p_history, p_hazard_logs = [], [] # Limpa históricos
        print(f"{LogColors.SUCCESS}Programa carregado com {len(p_program)} instruções. Pronto para iniciar.{LogColors.RESET}")
        with output_area: clear_output(wait=True) # Limpa a tabela anterior
    except (ValueError, KeyError) as e:
        print(f"{LogColors.ERROR}ERRO DE SINTAXE AO CARREGAR: {e}{LogColors.RESET}"); p_program = []

def execute_one_pipeline_cycle():
    """Executa um único ciclo de clock do pipeline."""
    global p_program, p_pc, p_cycle, p_stalls, p_mem_stalls, p_pipeline_stages, p_history, p_hazard_logs, regs, mem, cache

    # Pega as configurações atuais da UI
    hazard_mode = hazard_mode_selector.value
    cache_mode = cache_selector.value
    miss_penalty = cache_penalty_input.value

    # Condição de término: pipeline vazio E todas as instruções buscadas
    if p_pc >= len(p_program) and all(s is None for s in p_pipeline_stages.values()):
        finalize_pipeline_simulation(); return True

    p_cycle += 1; stall_this_cycle = False

    # Lógica de Stall de Memória (cache miss)
    if p_mem_stalls > 0:
        p_mem_stalls -= 1
        p_hazard_logs.append(f"Ciclo {p_cycle}: 🟡 STALL DE MEMÓRIA (Cache Miss)")
        p_history.append({ 'Ciclo': p_cycle, 'IF': "STALL", 'ID': "STALL", 'EX': "STALL", 'MEM_WB': format_instruction(p_pipeline_stages['MEM_WB'])})
        # Atualiza a exibição da tabela a cada ciclo
        df = pd.DataFrame(p_history).set_index('Ciclo');
        with output_area: clear_output(wait=True); display(df.fillna(""))
        return False # Não terminou

    prev_stages = deepcopy(p_pipeline_stages) # Copia profunda para estado anterior

    # --- Execução dos Estágios (ordem reversa) ---

    # ESTÁGIO 4: MEM_WB
    instr_wb = prev_stages['MEM_WB']
    if instr_wb:
        op = instr_wb['op']
        if op == "LW":
            data, status = cache.read(instr_wb['addr']) if cache_mode == 'enabled' else (mem.load_word(instr_wb['addr']), 'hit')
            if status == 'miss' and miss_penalty > 1: p_mem_stalls = miss_penalty - 1
            regs.write(instr_wb['rd'], data)
        elif op == "SW":
            _, status = cache.write(instr_wb['addr'], regs.read(instr_wb['rs1'])) if cache_mode == 'enabled' else (mem.store_word(instr_wb['addr'], regs.read(instr_wb['rs1'])), 'hit')
            if status == 'miss' and miss_penalty > 1: p_mem_stalls = miss_penalty - 1
        elif 'result' in instr_wb: regs.write(instr_wb['rd'], instr_wb['result'])

    # ESTÁGIO 3: EX
    instr_ex = prev_stages['EX']; flush_if_id = False
    if instr_ex:
        op = instr_ex['op']
        if op == 'BEQ' and regs.read(instr_ex['rs1']) == regs.read(instr_ex['rs2']):
            p_pc = instr_ex['target']; flush_if_id = True; p_hazard_logs.append(f"Ciclo {p_cycle}: 🔴 CONTROL HAZARD (BEQ). Desvio tomado, flush IF/ID.")
        elif op == 'J':
            p_pc = instr_ex['target']; flush_if_id = True; p_hazard_logs.append(f"Ciclo {p_cycle}: 🔴 CONTROL HAZARD (J). Desvio tomado, flush IF/ID.")
        else:
            val1, val2 = regs.read(instr_ex.get('rs1',0)), regs.read(instr_ex.get('rs2',0)) if 'rs2' in instr_ex else instr_ex.get('imm',0)
            if hazard_mode == 'forwarding':
                instr_mem = prev_stages['MEM_WB'] # Fwd MEM -> EX
                if instr_mem and 'result' in instr_mem and 'rd' in instr_mem and instr_mem['op'] != 'SW':
                     fwd_reg = instr_mem['rd']
                     if fwd_reg != 0 and fwd_reg == instr_ex.get('rs1'): val1 = instr_mem['result']; p_hazard_logs.append(f"Ciclo {p_cycle}: 🟢 FWD(MEM->EX) x{fwd_reg}")
                     if 'rs2' in instr_ex and fwd_reg != 0 and fwd_reg == instr_ex.get('rs2'): val2 = instr_mem['result']; p_hazard_logs.append(f"Ciclo {p_cycle}: 🟢 FWD(MEM->EX) x{fwd_reg}")
            if op in ["ADD", "ADDI"]: instr_ex['result'] = to_signed32(val1) + to_signed32(val2)
            elif op == "SUB": instr_ex['result'] = to_signed32(val1) - to_signed32(val2)
            # Adicione outras operações da ALU aqui (MUL, DIV, etc.) se necessário
        p_pipeline_stages['MEM_WB'] = instr_ex
    else: p_pipeline_stages['MEM_WB'] = None

    # ESTÁGIO 2: ID
    instr_id = prev_stages['ID']
    if instr_id:
        src_regs = {instr_id.get('rs1'), instr_id.get('rs2')} - {None, 0}
        instr_ex_prev = prev_stages['EX']
        if instr_ex_prev and 'rd' in instr_ex_prev and instr_ex_prev['rd'] != 0 and instr_ex_prev['rd'] in src_regs:
            hazard_reg = instr_ex_prev['rd']
            if instr_ex_prev['op'] == 'LW':
                if hazard_mode != 'none': stall_this_cycle = True; p_hazard_logs.append(f"Ciclo {p_cycle}: 🔴 HAZARD DE LOAD-USE x{hazard_reg}. Ação: STALL.")
                else: p_hazard_logs.append(f"Ciclo {p_cycle}: 🔴 HAZARD DE LOAD-USE x{hazard_reg}. 🟡 Ação: Nenhuma.")
            else:
                if hazard_mode == 'stall': stall_this_cycle = True; p_hazard_logs.append(f"Ciclo {p_cycle}: 🟡 HAZARD DE DADOS x{hazard_reg}. Ação: STALL.")
                elif hazard_mode == 'none': p_hazard_logs.append(f"Ciclo {p_cycle}: 🔴 HAZARD DE DADOS x{hazard_reg}. 🟡 Ação: Nenhuma.")

    # AVANÇO DO PIPELINE
    if stall_this_cycle: p_stalls += 1; p_pipeline_stages['EX'] = None
    elif flush_if_id: p_pipeline_stages['EX'] = prev_stages['ID']; p_pipeline_stages['ID'] = p_pipeline_stages['IF'] = None
    else:
        p_pipeline_stages['EX'] = prev_stages['ID']; p_pipeline_stages['ID'] = prev_stages['IF']
        if p_pc < len(p_program): p_pipeline_stages['IF'] = p_program[p_pc]; p_pc += 1
        else: p_pipeline_stages['IF'] = None

    p_history.append({ 'Ciclo': p_cycle, 'IF': format_instruction(p_pipeline_stages['IF']), 'ID': format_instruction(p_pipeline_stages['ID']), 'EX': f"*{format_instruction(p_pipeline_stages['EX'])}" if stall_this_cycle else format_instruction(p_pipeline_stages['EX']), 'MEM_WB': format_instruction(p_pipeline_stages['MEM_WB'])})

    # Atualiza a tabela na UI a cada ciclo
    df = pd.DataFrame(p_history).set_index('Ciclo')
    with output_area: clear_output(wait=True); display(df.fillna(""))
    show_registers(regs) # Atualiza regs também

    return False # Sinaliza que a execução não terminou

def finalize_pipeline_simulation():
    """Imprime as estatísticas finais após a conclusão do pipeline."""
    global p_program, p_cycle, p_stalls, p_history, p_hazard_logs
    if p_hazard_logs: print(f"\n{LogColors.INFO}--- Detalhes de Hazards e Stalls ---{LogColors.RESET}"); [print(log) for log in p_hazard_logs]
    print(f"\n{LogColors.INFO}Total de Ciclos (Pipeline):{LogColors.RESET} {p_cycle}")
    print(f"{LogColors.INFO}Número de Stalls (Data Hazard):{LogColors.RESET} {p_stalls}")
    print(f"{LogColors.INFO}Total de Ciclos (Ciclo Único seria):{LogColors.RESET} {len(p_program)}")
    speedup = len(p_program) / p_cycle if p_cycle > 0 else 0
    print(f"{LogColors.SUCCESS}Speedup (vs Ciclo Único): {speedup:.2f}x{LogColors.RESET}")
    print(f"\n{LogColors.SUCCESS}Fim do programa.{LogColors.RESET}")

print(f"{LogColors.SUCCESS}Funções de simulação da Parte Extra (com Step-by-Step) carregadas.{LogColors.RESET}")

# Conclusões

## Codigos dos testes

In [ ]:
codigo_exp1 = """# --- Teste para as Partes 1 & 2 ---

# Setup de valores
ADDI x1, x0, 100
ADDI x2, x0, 10
ADDI x3, x0, 7

# Teste de operações de inteiros
ADD x4, x1, x2  # x4 = 100 + 10 = 110
SUB x5, x1, x2  # x5 = 100 - 10 = 90
MUL x6, x2, x3  # x6 = 10 * 7 = 70
DIV x7, x1, x2  # x7 = 100 / 10 = 10

# Teste de Overflow (deve falhar e continuar)
ADDI x8, x0, 2147483647
ADD  x9, x8, x8

# Teste de Divisão por Zero (deve falhar e continuar)
DIV x10, x1, x0

# Teste de Ponto Flutuante
# Carrega os bits de 2.5 (1075838976) e 1.5 (1069547520)
ADDI x11, x0, 1075838976
ADDI x12, x0, 1069547520
ADD.S x13, x11, x12 # x13 = 2.5 + 1.5 = 4.0
MUL.S x14, x11, x12 # x14 = 2.5 * 1.5 = 3.75"""

In [ ]:
codigo_exp2 = """# --- Teste da Parte 3: Pipeline Básico ---

# Sequência de 5 instruções independentes
ADDI x1, x0, 1
ADDI x2, x0, 2
ADDI x3, x0, 3
ADDI x4, x0, 4
ADDI x5, x0, 5"""

In [ ]:
codigo_exp3 = """# --- Teste da Parte 4: Data Hazards ---

# Hazard de Dados (EX -> ID)
ADDI x1, x0, 10
ADDI x2, x0, 20
ADD  x3, x1, x2  # x3 é escrito aqui
SUB  x4, x3, x5  # x3 é lido imediatamente -> HAZARD!

# Hazard de Load-Use (LW -> EX)
ADDI x10, x0, 100
SW   x10, 40     # Armazena 100 no endereço 40
LW   x6, 40      # x6 é escrito com o valor da memória aqui
ADD  x7, x6, x1  # x6 é lido imediatamente -> HAZARD de LW!"""

In [ ]:
codigo_exp4 = """# --- Teste da Parte 5: Cache ---
# Cache: 8 conjuntos, 2 vias. Endereços 10, 18 e 26 mapeiam para o conjunto 2.

ADDI x1, x0, 100
ADDI x2, x0, 200

# 1. Miss de Compulsão e Hit Temporal
SW x1, 10   # MISS
LW x3, 10   # HIT

# 2. Miss de Conflito (LRU vs Random pode variar)
SW x2, 18   # MISS no conjunto 2
SW x1, 26   # MISS no conjunto 2 (pode expulsar 10 ou 18)

# 3. Teste da Política de Substituição
LW x4, 10   # Será HIT ou MISS? Depende se o bloco foi expulso."""

In [ ]:
codigo_exp5 = """# --- Teste Final: Desvios e Speedup ---
# Um loop simples que soma 10, 5 vezes.
    ADDI x1, x0, 5      # Contador do loop
    ADDI x2, x0, 0      # Acumulador (resultado)
    ADDI x3, x0, 10     # Valor a ser somado

loop:
    ADD  x2, x2, x3     # Acumula o valor
    ADDI x1, x1, -1     # Decrementa o contador
    BEQ  x1, x0, end    # Se contador for zero, vai para 'end'
    J    loop           # Pula de volta para o início do loop

end:
    SW x2, 50           # Salva o resultado final (50) na memória"""

In [ ]:
code_benchmark="""# --- Benchmark de Loop (Tese do Speedup) ---
# Executa um loop simples 100 vezes para demonstrar
# a vantagem de desempenho do pipeline em programas longos.

    ADDI x1, x0, 100    # Contador do loop (agora 100 iterações)
    ADDI x2, x0, 0      # Acumulador (resultado)
    ADDI x3, x0, 10     # Valor a ser somado

loop:
    # --- Corpo do Loop (3 instruções) ---
    ADD  x2, x2, x3     # Acumula o valor (1 ciclo no pipeline ideal)
    ADDI x1, x1, -1     # Decrementa o contador (1 ciclo)
    BEQ  x1, x0, end    # Verifica se terminou (1 ciclo + penalidade de desvio)
    # --- Fim do Corpo ---

    J    loop           # Pula de volta (1 ciclo + penalidade de desvio)

end:
    # Instrução final após o loop
    SW x2, 50           # Salva o resultado final (1000) na memória"""

## <strong>Experimento 1: Validação das Partes 1 & 2 (Operações e Ciclo Único)</strong>

<strong>Objetivo:</strong> Validar todas as operações aritméticas de inteiros e ponto flutuante, incluindo a detecção de erros (overflow, divisão por zero), no modo de ciclo único.

<strong>Código Assembly:</strong>


In [ ]:
display(create_copy_button(codigo_exp1, button_id=1))

Configurações do Simulador:
- <strong>Simular:</strong> ```Ciclo Único```

Análise dos Resultados Esperados:
- <strong>Log de Execução</strong>: Você deverá ver mensagens de <strong>"Sucesso!"</strong> para todas as operações válidas. As instruções ```ADD x9, x8, x8``` e ```DIV x10, x1, x0``` deverão mostrar mensagens de ```ERRO: Overflow...``` e ```ERRO: Divisão por zero```, respectivamente. O programa não deve parar.
- <strong>Registradores:</strong> Ao clicar em "Show Registers", verifique se:
  - ```x4```=110, ```x5```=90, ```x6```=70, ```x7```=10.
  - ```x9``` e ```x10``` permanecem ```0```, pois suas operações falharam.
  - Na coluna "Valor FP", ```x13``` deve ser ```4.0000``` e ```x14``` deve ser ```3.7500```.

## <strong>Experimento 2: Validação da Parte 3 (Pipeline Básico)</strong>

<strong>Objetivo:</strong> Demonstrar o funcionamento do pipeline com 4 estágios, mostrando a tabela de evolução e comparando o número de ciclos com o modelo de ciclo único.

<strong>Código Assembly:</strong>


In [ ]:
display(create_copy_button(codigo_exp2, button_id=2))

Configurações do Simulador:
- <strong>Simular:</strong> ```Pipeline (Completo)```
- <strong>Hazards:</strong> ```Sem tratamento de hazards``` (para evitar interferência)
- <strong>Memória:</strong> ```Sem Cache```

Análise dos Resultados Esperados:
- <strong>Tabela do Pipeline:</strong> A tabela exibida na área de output deve mostrar o fluxo diagonal das instruções, preenchendo o pipeline nos primeiros 4 ciclos, executando em capacidade máxima, e depois esvaziando.

- <strong>Contagem de Ciclos:</strong> O log final deve mostrar um ```Total de Ciclos (Pipeline)``` de 8 (ou 9, dependendo da condição de parada do loop). Isso valida a fórmula ```Nº de Instruções + Nº de Estágios - 1 = 5 + 4 - 1 = 8```. O ```Total de Ciclos (Ciclo Único seria)``` será 5.

## <strong>Experimento 3: Validação da Parte 4 e Ponto Extra (Análise de Hazards)</strong>

<strong>Objetivo:</strong> Demonstrar a detecção de data hazards, comparar a eficácia das soluções com stalls e forwarding, e responder à pergunta: <strong>*"Qual a diferença entre a solução com stalls e com forwarding?"*</strong>

<strong>Código Assembly:</strong>


In [ ]:
display(create_copy_button(codigo_exp3, button_id=3))

Configurações do Simulador (Execute 3 vezes):
- <strong>Run 1 (Stall):</strong> Selecione ```Pipeline```, ```Stall em hazards```, ```Sem Cache```.
- <strong>Run 2 (Forwarding):</strong> Mude apenas para ```Forwarding```.
- <strong>Run 3 (Sem Tratamento):</strong> Mude apenas para ```Sem tratamento de hazards```.

Análise dos Resultados Esperados:

- <strong>Run 1 (Stall):</strong> O log de "Detalhes de Hazards" mostrará <strong>2 hazards</strong> e a ação de <strong>STALL</strong> para ambos. O ```Número de Stalls``` será > 0 e o ```Total de Ciclos será alto```. Os resultados nos registradores (```x4``` e ```x7```) estarão <strong>corretos</strong>.

- <strong>Run 2 (Forwarding):</strong> O log mostrará uma ação de <strong>FORWARDING</strong> para o primeiro hazard e um <strong>STALL OBRIGATÓRIO</strong> para o hazard de Load-Use. O ```Número de Stalls``` será menor que no Run 1, e o ```Total de Ciclos``` também será menor. Os resultados estarão <strong>corretos</strong>.

- <strong>Run 3 (Sem Tratamento):</strong> O log mostrará a <strong>detecção dos 2 hazards<strong>, mas a ação será </strong>"Nenhuma"</strong>. O ```Número de Stalls``` será 0, e o ```Total de Ciclos``` será o menor de todos. No entanto, os valores finais de ```x4``` e ```x7``` estarão <strong>incorretos</strong>, pois as operações usaram dados antigos.

- <strong>Resposta ao Ponto Extra:</strong> A diferença é que a solução com *stalls* para o pipeline, perde ciclos ao colocar muitos stalls, enquanto o *forwarding* resolve a dependência sem parar o pipeline na maioria dos casos, resultando em menos stalls e um menor tempo de execução total, mantendo a correção do programa.

## <strong>Experimento 4: Validação da Parte 5 e Ponto Extra (Cache)</strong>

<strong>Objetivo:</strong> Demonstrar o funcionamento da cache, calcular hits/misses, e responder às perguntas: <strong>*"Como a taxa de acerto da cache afetou o tempo de execução?"*</strong> e <strong>*"Comparando diferentes políticas de substituição de cache (LRU vs Random)."*</strong>

<strong>Código Assembly:</strong>


In [ ]:
display(create_copy_button(codigo_exp4, button_id=4))

Configurações do Simulador (Execute 3 vezes):
- <strong>Run 1 (LRU com Penalidade):</strong> Selecione ```Pipeline```, ```Forwarding```, ```Com Cache```, ```LRU```, ```Penalidade Miss: 10```.

- <strong>Run 2 (Random com Penalidade):</strong> Mude apenas para ```Random```.

- <strong>Run 3 (LRU com Alta Penalidade):</strong> Mude para ```LRU``` e ```Penalidade Miss: 50```.

Análise dos Resultados Esperados:

- <strong>Run 1 (LRU):</strong> Anote a taxa de acertos e o ```Total de Ciclos```. A última leitura (```LW x4, 10```) provavelmente será um <strong>miss</strong>, pois o acesso a ```18``` e ```26``` fez com que o bloco ```10``` se tornasse o menos recentemente usado.

- <strong>Run 2 (Random):</strong> A taxa de acertos pode ser <strong>diferente</strong> da de LRU. A última leitura (```LW x4, 10```) pode ser um <strong>hit</strong> se a política ```Random``` tiver sorte e expulsar o bloco ```18``` em vez do ```10```. Isso demonstra a natureza não-determinística da política Random.

- <strong>Run 3 (Alta Penalidade):</strong> A taxa de acertos será a mesma do Run 1, mas o ```Total de Ciclos``` será <strong>significativamente maior</strong>.

- <strong>Respostas aos Pontos Extras:</strong>

  - <strong>Impacto da Taxa de Acerto:</strong> Comparando Run 1 e Run 3, você mostra que, embora a taxa de acerto não mude, o custo dos misses (a penalidade) aumenta drasticamente o tempo de execução, provando que uma alta taxa de acerto é crucial para o desempenho.

  - <strong>Políticas de Substituição:</strong> Comparando Run 1 e Run 2, você mostra que a política de substituição pode levar a diferentes taxas de acerto para o mesmo padrão de acesso.



## <strong>Experimento 5: Validação de Pontos Extras (Control Hazards e Speedup Final)</strong>

<strong>Objetivo:</strong> Demonstrar a simulação de desvios e control hazards, e medir o speedup final do sistema otimizado em relação ao ciclo único.

<strong>Código Assembly:</strong>


In [ ]:
display(create_copy_button(codigo_exp5, button_id=5))

Configurações do Simulador (Execute 2 vezes):

- <strong>Run 1 (Baseline):</strong> Selecione ```Ciclo Único```.

- <strong>Run 2 (Otimizado):</strong> Selecione ```Pipeline```, ```Forwarding```, ```Com Cache```, ```LRU```, ```Penalidade Miss: 10```.

Análise dos Resultados Esperados:

- <strong>Run 2 (Otimizado):</strong> O log de "Detalhes de Hazards" mostrará múltiplos ```CONTROL HAZARD``` ```(BEQ)``` e ```(J)```, indicando que os desvios foram detectados e o pipeline foi esvaziado (flushed), o que adiciona ciclos de penalidade.

- <strong>Registradores e Memória:</strong> Em ambos os runs, o resultado final deve ser <strong>correto</strong>. O registrador ```x2``` deve terminar com o valor ```50```, e o endereço de memória ```50``` também conterá ```50```.

- <strong>Resposta ao Ponto Extra (Speedup):</strong> Pegue o ```Total de Ciclos``` do Run 1 (Ciclo Único) e do Run 2 (Pipeline). Calcule o <strong>Speedup = Ciclos_Único / Ciclos_Pipeline</strong>. O log do Run 2 já fará isso para você. Este valor quantifica o ganho de desempenho total de todas as otimizações (pipeline, forwarding, cache) em relação ao design mais simples, mesmo considerando as penalidades de hazards de controle e cache misses.

## <strong>Experimento 6: Benchmark de Loop</strong>

<strong>Objetivo:</strong> Demonstrar speedup do pipeline em relação ao ciclo único.

<strong>Código Assembly:</strong>


In [ ]:
display(create_copy_button(codigo_benckmark, button_id=6))

Configurações do Simulador (Execute 2 vezes):

- <strong>Run 1 (Baseline):</strong> Selecione ```Ciclo Único```.

- <strong>Run 2 (Otimizado):</strong> Selecione ```Pipeline```, ```Forwarding```, ```Com Cache```, ```LRU```, ```Penalidade Miss: 10```.

Análise dos Resultados Esperados:

- <strong>Run 2 (Otimizado):</strong> O log de "Detalhes de Hazards" mostrará múltiplos ```CONTROL HAZARD``` ```(BEQ)``` e ```(J)```, indicando que os desvios foram detectados e o pipeline foi esvaziado (flushed), o que adiciona ciclos de penalidade.

- <strong>Registradores e Memória:</strong> Em ambos os runs, o resultado final deve ser <strong>correto</strong>. O registrador ```x2``` deve terminar com o valor ```50```, e o endereço de memória ```50``` também conterá ```50```.

- <strong>Resposta ao Ponto Extra (Speedup):</strong> Pegue o ```Total de Ciclos``` do Run 1 (Ciclo Único) e do Run 2 (Pipeline). Calcule o <strong>Speedup = Ciclos_Único / Ciclos_Pipeline</strong>. O log do Run 2 já fará isso para você. Este valor quantifica o ganho de desempenho total de todas as otimizações (pipeline, forwarding, cache) em relação ao design mais simples, mesmo considerando as penalidades de hazards de controle e cache misses.